# autograd
用Tensor训练网络很方便，但从上一小节最后的线性回归例子来看，反向传播过程需要手动实现。这对于像线性回归等较为简单的模型来说，还可以应付，但实际使用中经常出现非常复杂的网络结构，此时如果手动实现反向传播，不仅费时费力，而且容易出错，难以检查。torch.autograd就是为方便用户使用，而专门开发的一套自动求导引擎，它能够根据输入和前向传播过程自动构建计算图，并执行反向传播。

计算图(Computation Graph)是现代深度学习框架如PyTorch和TensorFlow等的核心，其为高效自动求导算法——反向传播(Back Propogation)提供了理论支持，了解计算图在实际写程序过程中会有极大的帮助。本节将涉及一些基础的计算图知识，但并不要求读者事先对此有深入的了解。关于计算图的基础知识推荐阅读Christopher Olah的文章.


##  requires_grad

PyTorch在autograd模块中实现了计算图的相关功能，autograd中的核心数据结构是Variable。从v0.4版本起，Variable和Tensor合并。我们可以认为需要求导(requires_grad)的tensor即Variable. autograd记录对tensor的操作记录用来构建计算图。

Variable提供了大部分tensor支持的函数，但其不支持部分`inplace`函数，因这些函数会修改tensor自身，而在反向传播中，variable需要缓存原来的tensor来计算反向传播梯度。如果想要计算各个Variable的梯度，只需调用根节点variable的`backward`方法，autograd会自动沿着计算图反向传播，计算每一个叶子节点的梯度。

`variable.backward(gradient=None, retain_graph=None, create_graph=None)`主要有如下参数：

- grad_variables：形状与variable一致，对于`y.backward()`，grad_variables相当于链式法则${dz \over dx}={dz \over dy} \times {dy \over dx}$中的$\textbf {dz} \over \textbf {dy}$。grad_variables也可以是tensor或序列。
- retain_graph：反向传播需要缓存一些中间结果，反向传播之后，这些缓存就被清空，可通过指定这个参数不清空缓存，用来多次反向传播。
- create_graph：对反向传播过程再次构建计算图，可通过`backward of backward`实现求高阶导数。

上述描述可能比较抽象，如果没有看懂，不用着急，会在本节后半部分详细介绍，下面先看几个例子。

In [30]:
from __future__ import print_function
import torch as t

In [31]:
# 注意在创建Tensor的时候需要指定requires_grad
a=t.randn(3,4,requires_grad=True)

#或者
a=t.randn(3,4).requires_grad_()

#或者
a=t.randn(3,4)
a.requires_grad=True
a

tensor([[-0.8873, -0.4227, -1.8543, -0.6434],
        [-0.2812,  1.3114, -0.0459, -0.2372],
        [ 1.2774,  0.8956, -0.3374, -0.5551]], requires_grad=True)

In [32]:
b=t.zeros(3,4).requires_grad_()
b

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]], requires_grad=True)

In [33]:
c=a+b
c

tensor([[-0.8873, -0.4227, -1.8543, -0.6434],
        [-0.2812,  1.3114, -0.0459, -0.2372],
        [ 1.2774,  0.8956, -0.3374, -0.5551]], grad_fn=<AddBackward0>)

In [34]:
d=c.sum()
d.backward()

In [35]:
d # d还是一个requires_grad=True的tensor,对它的操作需要慎重
d.requires_grad

True

In [36]:
a.grad

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])

In [37]:
# 此处虽然没有指定c需要求导，但c依赖于a，而a需要求导，
# 因此c的requires_grad属性会自动设为True
a.requires_grad,b.requires_grad,c.requires_grad

(True, True, True)

In [38]:
# 由用户创建的variable属于叶子节点，对应的grad_fn是None,
a.is_leaf,b.is_leaf,c.is_leaf

(True, True, False)

In [39]:
# c.grad是None, 因c不是叶子节点，它的梯度是用来计算a的梯度
# 所以虽然c.requires_grad = True,但其梯度计算完之后即被释放
c.grad_fn,c.grad is None

(<AddBackward0 at 0x2ad8f658cf8>, True)

计算下面这个函数的导函数：
$$
y = x^2\bullet e^x
$$
它的导函数是：
$$
{dy \over dx} = 2x\bullet e^x + x^2 \bullet e^x
$$
来看看autograd的计算结果与手动求导计算结果的误差。

In [40]:
def f(x):
  y=x**2*t.exp(x)
  return y

def gradf(x):
  dx=2*x*t.exp(x)+x**2*t.exp(x)
  return dx

In [41]:
x=t.randn(3,4,requires_grad=True)
y=f(x)
y

tensor([[4.3338e-01, 3.8330e+00, 1.0013e+01, 4.1377e-01],
        [1.8173e-01, 4.4784e-01, 9.8027e+00, 3.1920e-01],
        [3.4684e-01, 6.1543e-02, 3.2282e-01, 6.4124e+01]],
       grad_fn=<MulBackward0>)

In [42]:
y.backward(t.ones(y.size())) #gradient形状与y一致
x.grad

tensor([[ -0.2896,  10.6843,  23.3904,  -0.3159],
        [  1.2008,  -0.2683,  22.9792,   1.7349],
        [ -0.3877,  -0.3684,  -0.4076, 117.4412]])

In [43]:
gradf(x)

tensor([[ -0.2896,  10.6843,  23.3904,  -0.3159],
        [  1.2008,  -0.2683,  22.9792,   1.7349],
        [ -0.3877,  -0.3684,  -0.4076, 117.4412]], grad_fn=<AddBackward0>)

## 计算图
PyTorch中`autograd`的底层采用了计算图，计算图是一种特殊的有向无环图（DAG），用于记录算子与变量之间的关系。一般用矩形表示算子，椭圆形表示变量。如表达式$ \textbf {z = wx + b}$可分解为$\textbf{y = wx}$和$\textbf{z = y + b}$，其计算图如图3-3所示，图中`MUL`，`ADD`都是算子，$\textbf{w}$，$\textbf{x}$，$\textbf{b}$即变量。

![图3-3:computation graph](http://localhost:8888/notebooks/pytorch/pytorch-book-master/chapter3-Tensor%E5%92%8Cautograd/imgs/com_graph.svg)

如上有向无环图中，$\textbf{X}$和$\textbf{b}$是叶子节点（leaf node），这些节点通常由用户自己创建，不依赖于其他变量。$\textbf{z}$称为根节点，是计算图的最终目标。利用链式法则很容易求得各个叶子节点的梯度。
$${\partial z \over \partial b} = 1,\space {\partial z \over \partial y} = 1\\
{\partial y \over \partial w }= x,{\partial y \over \partial x}= w\\
{\partial z \over \partial x}= {\partial z \over \partial y} {\partial y \over \partial x}=1 * w\\
{\partial z \over \partial w}= {\partial z \over \partial y} {\partial y \over \partial w}=1 * x\\
$$
而有了计算图，上述链式求导即可利用计算图的反向传播自动完成，其过程如图3-4所示。

![图3-4：计算图的反向传播](http://localhost:8888/notebooks/pytorch/pytorch-book-master/chapter3-Tensor%E5%92%8Cautograd/imgs/com_graph_backward.svg)


在PyTorch实现中，autograd会随着用户的操作，记录生成当前variable的所有操作，并由此建立一个有向无环图。用户每进行一个操作，相应的计算图就会发生改变。更底层的实现中，图中记录了操作`Function`，每一个变量在图中的位置可通过其`grad_fn`属性在图中的位置推测得到。在反向传播过程中，autograd沿着这个图从当前变量（根节点$\textbf{z}$）溯源，可以利用链式求导法则计算所有叶子节点的梯度。每一个前向传播操作的函数都有与之对应的反向传播函数用来计算输入的各个variable的梯度，这些函数的函数名通常以`Backward`结尾。下面结合代码学习autograd的实现细节。

In [44]:
x=t.ones(1)
b=t.rand(1,requires_grad=True)
w=t.rand(1,requires_grad=True)
y=w*x
z=y+b

In [45]:
x.requires_grad,b.requires_grad,w.requires_grad

(False, True, True)

In [46]:
# 虽然未指定y.requires_grad为True，但由于y依赖于需要求导的w
# 故而y.requires_grad为True
y.requires_grad

True

In [47]:
x.is_leaf,w.is_leaf,b.is_leaf

(True, True, True)

In [48]:
y.is_leaf,z.is_leaf

(False, False)

In [49]:
# grad_fn可以查看这个variable的反向传播函数，
# z是add函数的输出，所以它的反向传播函数是AddBackward
z.grad_fn

In [50]:
# next_functions保存grad_fn的输入，是一个tuple，tuple的元素也是Function
# 第一个是y，它是乘法(mul)的输出，所以对应的反向传播函数y.grad_fn是MulBackward
# 第二个是b，它是叶子节点，由用户创建，grad_fn为None，但是有
z.grad_fn.next_functions

((<MulBackward0 at 0x2ad8f659278>, 0), (<AccumulateGrad at 0x2ad8f659198>, 0))

In [51]:
# variable的grad_fn对应着和图中的function相对应
z.grad_fn.next_functions[0][0]==y.grad_fn

True

In [52]:
# 第一个是w，叶子节点，需要求导，梯度是累加的
# 第二个是x，叶子节点，不需要求导，所以为None
y.grad_fn.next_functions

((<AccumulateGrad at 0x2ad8f659470>, 0), (None, 0))

In [53]:
# 叶子节点的grad_fn是None
w.grad_fn,x.grad_fn

(None, None)

计算w的梯度的时候，需要用到x的数值(${\partial y\over \partial w} = x $)，这些中间数值在前向过程中会保存成buffer，在计算完梯度之后会自动清空。为了能够多次反向传播需要指定`retain_graph`来保留这些buffer。

In [54]:
# 使用retain_graph来保存buffer
z.backward(retain_graph=True)
w.grad

tensor([1.])

In [55]:
# 多次反向传播，梯度累加，这也就是w中AccumulateGrad标识的含义
z.backward()
w.grad

tensor([2.])

In [101]:
x=t.arange(0,3,dtype=t.float32,requires_grad=True)
y=x**2+x*2
z=y.sum()
z.backward()## 从z开始反向传播 ,z是个标量
x.grad

tensor([2., 4., 6.])

In [102]:
y=x**2+x*2
z=y.sum()
z.backward()
x.grad

tensor([ 4.,  8., 12.])

PyTorch使用的是动态图，它的计算图在每次前向传播时都是从头开始构建，所以它能够使用Python控制语句（如for、if等）根据需求创建计算图。这点在自然语言处理领域中很有用，它意味着你不需要事先构建所有可能用到的图的路径，图在运行时才构建。

In [56]:
def abs(x):
  if x.data[0]>0:
    return x
  else: 
    return -x
 
x=t.ones(1,requires_grad=True)
y=abs(x)
y.backward()
x.grad

tensor([1.])

In [57]:
x=-1*t.ones(1)
x.requires_grad=True
y=abs(x)
y.backward()
x.grad

tensor([-1.])

In [64]:
def f(x):
    result=1
    for ii in x:
        if ii.item()>0:
            result=ii*result
    return result
  
x=t.arange(-2,4,dtype=t.float32,requires_grad=True)
y=f(x)
y.backward()
x.grad

tensor([0., 0., 0., 6., 3., 2.])

In [63]:
x

tensor([-1.], requires_grad=True)

变量的`requires_grad`属性默认为False，如果某一个节点requires_grad被设置为True，那么所有依赖它的节点`requires_grad`都是True。这其实很好理解，对于$ \textbf{x}\to \textbf{y} \to \textbf{z}$，x.requires_grad = True，当需要计算$\partial z \over \partial x$时，根据链式法则，$\frac{\partial z}{\partial x} = \frac{\partial z}{\partial y} \frac{\partial y}{\partial x}$，自然也需要求$ \frac{\partial z}{\partial y}$，所以y.requires_grad会被自动标为True. 



有些时候我们可能不希望autograd对tensor求导。认为求导需要缓存许多中间结构，增加额外的内存/显存开销，那么我们可以关闭自动求导。对于不需要反向传播的情景（如inference，即测试推理时），关闭自动求导可实现一定程度的速度提升，并节省约一半显存，因其不需要分配空间计算梯度。

In [66]:
x=t.ones(1,requires_grad=True)
w=t.rand(1,requires_grad=True)
y=x*w
# y依赖于w，而w.requires_grad = True
y.requires_grad,w.requires_grad,x.requires_grad

(True, True, True)

In [67]:
with t.no_grad():
    x=t.ones(1)
    w=t.rand(1,requires_grad=True)
    y=x*w
# y依赖于w和x，虽然w.requires_grad = True，但是y的requires_grad依旧为False
y.requires_grad,w.requires_grad,x.requires_grad

(False, True, False)

In [68]:
t.no_grad??

In [69]:
t.set_grad_enabled(False)
x=t.ones(1)
w=t.rand(1,requires_grad=True)
y=w*x
# y依赖于w和x，虽然w.requires_grad = True，但是y的requires_grad依旧为False
y.requires_grad,w.requires_grad,x.requires_grad

(False, True, False)

In [70]:
# 恢复默认配置
t.set_grad_enabled(True)

如果我们想要修改tensor的数值，但是又不希望被autograd记录，那么我么可以对tensor.data进行操作

In [72]:
a=t.ones(3,4,requires_grad=True)
b=t.ones(3,4,requires_grad=True)
c=a*b

a.data #还是一个tensor

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])

In [73]:
a.data.requires_grad #但是已经时独立于计算图之外的

False

In [74]:
d=a.data.sigmoid_()
d.requires_grad

False

In [75]:
a

tensor([[0.7311, 0.7311, 0.7311, 0.7311],
        [0.7311, 0.7311, 0.7311, 0.7311],
        [0.7311, 0.7311, 0.7311, 0.7311]], requires_grad=True)

如果我们希望对tensor，但是又不希望被记录, 可以使用tensor.data 或者tensor.detach()

In [76]:
a.requires_grad

True

In [77]:
# 近似于 tensor=a.data, 但是如果tensor被修改，backward可能会报错
tensor=a.detach()
tensor.requires_grad

False

In [78]:
## 统计tensor的一些指标，不希望被记录
mean=tensor.mean()
std=tensor.std()
maximum=tensor.max()

In [79]:
tensor[0]=1
# 下面会报错：　RuntimeError: one of the variables needed for gradient
#             computation has been modified by an inplace operation
#　因为 c=a*b, b的梯度取决于a，现在修改了tensor，其实也就是修改了a，梯度不再准确
# c.sum().backward() 

In [81]:
#c.sum().backward()

在反向传播过程中非叶子节点的导数计算完之后即被清空。若想查看这些变量的梯度，有两种方法：

* 使用autograd.grad函数
* 使用hook

autograd.grad和hook方法都是很强大的工具，更详细的用法参考官方api文档，这里举例说明基础的使用。推荐使用hook方法，但是在实际使用中应尽量避免修改grad的值。

In [86]:
x=t.ones(3,requires_grad=True)
w=t.rand(3,requires_grad=True)
y=w*x
# y依赖于w，而w.requires_grad = True
z=y.sum()
x.requires_grad,w.requires_grad,y.requires_grad

(True, True, True)

In [87]:
# 非叶子节点grad计算完之后自动清空，y.grad是None
z.backward()
(x.grad,w.grad,y.grad,z.grad)

(tensor([0.9194, 0.9469, 0.8179]), tensor([1., 1., 1.]), None, None)

In [88]:
# 第一种方法：使用grad获取中间变量的梯度
x=t.ones(3,requires_grad=True)
w=t.rand(3,requires_grad=True)
y=w*x
# y依赖于w，而w.requires_grad = True
z=y.sum()
t.autograd.grad(z,y)

(tensor([1., 1., 1.]),)

In [89]:
# 第二种方法：使用hook
# hook是一个函数，输入是梯度，不应该有返回值
def variable_hook(grad):
    print('y的梯度：',grad)
x=t.ones(3,requires_grad=True)
w=t.rand(3,requires_grad=True)
y=w*x
#注册hook
hook_handle=y.register_hook(variable_hook)
z=y.sum()
z.backward()

# 除非你每次都要用hook，否则用完之后记得移除hook
hook_handle.remove()

y的梯度： tensor([1., 1., 1.])


最后再来看看variable中grad属性和backward函数`grad_variables`参数的含义，这里直接下结论：

- variable $\textbf{x}$的梯度是目标函数${f(x)} $对$\textbf{x}$的梯度，$\frac{df(x)}{dx} = (\frac {df(x)}{dx_0},\frac {df(x)}{dx_1},...,\frac {df(x)}{dx_N})$，形状和$\textbf{x}$一致。
- 对于y.backward(grad_variables)中的grad_variables相当于链式求导法则中的$\frac{\partial z}{\partial x} = \frac{\partial z}{\partial y} \frac{\partial y}{\partial x}$中的$\frac{\partial z}{\partial y}$。z是目标函数，一般是一个标量，故而$\frac{\partial z}{\partial y}$的形状与variable $\textbf{y}$的形状一致。`z.backward()`在一定程度上等价于y.backward(grad_y)。`z.backward()`省略了grad_variables参数，是因为$z$是一个标量，而$\frac{\partial z}{\partial z} = 1$

In [91]:
x=t.arange(0,3,dtype=t.float32,requires_grad=True)
y=x**2+x*2
z=y.sum()
z.backward()## 从z开始反向传播 ,z是个标量
x.grad

tensor([2., 4., 6.])

In [92]:
x=t.arange(0.0,3.0,requires_grad=True)
y=x**2+x*2
z=y.sum()
y_gradient=t.Tensor([1,1,1])#dz/dy
y.backward(y_gradient) #y是个向量
x.grad

tensor([2., 4., 6.])

另外值得注意的是，只有对variable的操作才能使用autograd，如果对variable的data直接进行操作，将无法使用反向传播。除了对参数初始化，一般我们不会修改variable.data的值。

在PyTorch中计算图的特点可总结如下：

* autograd根据用户对variable的操作构建其计算图。对变量的操作抽象为Function。
* 对于那些不是任何函数(Function)的输出，由用户创建的节点称为叶子节点，叶子节点的grad_fn为None。叶子节点中需要求导的variable，具有AccumulateGrad标识，因其梯度是累加的。
* variable默认是不需要求导的，即requires_grad属性默认为False，如果某一个节点requires_grad被设置为True，那么所有依赖它的节点requires_grad都为True。
* variable的volatile属性默认为False，如果某一个variable的volatile属性被设为True，那么所有依赖它的节点volatile属性都为True。volatile属性为True的节点不会求导，volatile的优先级比requires_grad高。
* 多次反向传播时，梯度是累加的。反向传播的**中间缓存**会被清空，为进行多次反向传播需指定retain_graph=True来保存这些缓存。
* 非叶子节点的梯度计算完之后即被清空，可以使用autograd.grad或hook技术获取非叶子节点的值。
* variable的grad与data形状一致，应避免直接修改variable.data，因为对data的直接操作无法利用autograd进行反向传播
* 反向传播函数backward的参数grad_variables可以看成链式求导的中间结果，如果是标量，可以省略，默认为1
* PyTorch采用动态图设计，可以很方便地查看中间层的输出，动态的设计计算图结构。

这些知识不懂大多数情况下也不会影响对pytorch的使用，但是掌握这些知识有助于更好的理解pytorch，并有效的避开很多陷阱

## 扩展autograd
目前绝大多数函数都可以使用`autograd`实现反向求导，但如果需要自己写一个复杂的函数，不支持自动反向求导怎么办? 写一个`Function`，实现它的前向传播和反向传播代码，`Function`对应于计算图中的矩形， 它接收参数，计算并返回结果。下面给出一个例子。

```python

class Mul(Function):
                                                            
    @staticmethod
    def forward(ctx, w, x, b, x_requires_grad = True):
        ctx.x_requires_grad = x_requires_grad
        ctx.save_for_backward(w,x)
        output = w * x + b
        return output
        
    @staticmethod
    def backward(ctx, grad_output):
        w,x = ctx.saved_tensors
        grad_w = grad_output * x
        if ctx.x_requires_grad:
            grad_x = grad_output * w
        else:
            grad_x = None
        grad_b = grad_output * 1
        return grad_w, grad_x, grad_b, None
```

分析如下：

- 自定义的Function需要继承autograd.Function，没有构造函数`__init__`，forward和backward函数都是静态方法
- backward函数的输出和forward函数的输入一一对应，backward函数的输入和forward函数的输出一一对应
- backward函数的grad_output参数即t.autograd.backward中的`grad_variables`
- 如果某一个输入不需要求导，直接返回None，如forward中的输入参数x_requires_grad显然无法对它求导，直接返回None即可
- 反向传播可能需要利用前向传播的某些中间结果，需要进行保存，否则前向传播结束后这些对象即被释放

Function的使用利用Function.apply(variable)

In [103]:
from torch.autograd import Function
class MultiplyAdd(Function):
    @staticmethod
    def forward(ctx,w,x,b):
        ctx.save_for_backward(w,x)
        output=w*x+b
        return output
    
    @staticmethod
    def backward(ctx,grad_output):
        w,x=ctx.saved_tensors
        grad_w=grad_output*x
        grad_x=grad_output*w
        grad_b=grad_output*1
        return grad_w,grad_x,grad_b

In [104]:
x=t.ones(1)
w=t.rand(1,requires_grad=True)
b=t.rand(1,requires_grad=True)

#开始前向传播
z=MultiplyAdd.apply(w,x,b)

#print('开始反向传播')
# 调用MultiplyAdd.backward
# 输出grad_w, grad_x, grad_b
z.grad_fn.apply(t.ones(1))

(tensor([1.]), tensor([0.0143], grad_fn=<MulBackward0>), tensor([1.]))

之所以forward函数的输入是tensor，而backward函数的输入是variable，是为了实现高阶求导。backward函数的输入输出虽然是variable，但在实际使用时autograd.Function会将输入variable提取为tensor，并将计算结果的tensor封装成variable返回。在backward函数中，之所以也要对variable进行操作，是为了能够计算梯度的梯度（backward of backward）。下面举例说明，有关torch.autograd.grad的更详细使用请参照文档。

In [106]:
x=t.tensor([5.0],requires_grad=True)
y=x**2
grad_x=t.autograd.grad(y,x,create_graph=True)
grad_x

(tensor([10.], grad_fn=<MulBackward0>),)

In [107]:
grad_grad_x=t.autograd.grad(grad_x[0],x)
grad_grad_x

(tensor([2.]),)

这种设计虽然能让autograd具有高阶求导功能，但其也限制了Tensor的使用，因autograd中反向传播的函数只能利用当前已经有的Variable操作。这个设计是在0.2版本新加入的，为了更好的灵活性，也为了兼容旧版本的代码，PyTorch还提供了另外一种扩展autograd的方法。PyTorch提供了一个装饰器@once_differentiable，能够在backward函数中自动将输入的variable提取成tensor，把计算结果的tensor自动封装成variable。有了这个特性我们就能够很方便的使用numpy/scipy中的函数，操作不再局限于variable所支持的操作。但是这种做法正如名字中所暗示的那样只能求导一次，它打断了反向传播图，不再支持高阶求导。

上面所描述的都是新式Function，还有个legacy Function，可以带有__init__方法，forward和backwad函数也不需要声明为@staticmethod，但随着版本更迭，此类Function将越来越少遇到，在此不做更多介绍。

此外在实现了自己的Function之后，还可以使用gradcheck函数来检测实现是否正确。gradcheck通过数值逼近来计算梯度，可能具有一定的误差，通过控制eps的大小可以控制容忍的误差。 关于这部份的内容可以参考github上开发者们的讨论^3。

In [108]:
t.__version__

'1.0.0'

In [110]:
class Sigmoid(Function):
    @staticmethod
    def forward(ctx,x):
        output=1/(1+t.exp(-x))
        ctx.save_for_backward(output)
        return output
    
    @staticmethod
    def backward(ctx,grad_output):
        output,=ctx.saved_tensors
        grad_x=output*(1-output)*grad_output
        return grad_x

In [111]:
# 采用数值逼近方式检验计算梯度的公式对不对
test_input=t.randn(3,4,requires_grad=True)
t.autograd.gradcheck(Sigmoid.apply,(test_input,),eps=1e-3)

c:\users\liyajie\appdata\local\programs\python\python36\lib\site-packages\torch\autograd\gradcheck.py:170: UserWarning: At least one of the inputs that requires gradient is not of double precision floating point. This check will likely fail if all the inputs are not of double precision floating point. 
  'At least one of the inputs that requires gradient '


True

In [113]:
def f_sigmoid(x):
    y = Sigmoid.apply(x)
    y.backward(t.ones(x.size()))
    
def f_naive(x):
    y =  1/(1 + t.exp(-x))
    y.backward(t.ones(x.size()))
    
def f_th(x):
    y = t.sigmoid(x)
    y.backward(t.ones(x.size()))
    
x=t.randn(100, 100, requires_grad=True)
%timeit -n 100 f_sigmoid(x)
%timeit -n 100 f_naive(x)
%timeit -n 100 f_th(x)

215 µs ± 30.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
191 µs ± 2.31 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
164 µs ± 12.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


显然f_sigmoid要比单纯利用autograd加减和乘方操作实现的函数快不少，因为f_sigmoid的backward优化了反向传播的过程。另外可以看出系统实现的built-in接口(t.sigmoid)更快。